# Gmail analysis
#### Study of 13 years of Gmail messages

I learned recently that Google allows its user to download metadata for all the messages sent and received through Gmail. This leads to interesting insights to be found in the data: what are the most common people I have been in touch with? What days of the week or time of the day are have the highest trafic? In order to explore these and other questions I decided to request my data and perform the present analysis.

The first step is [requesting the data](https://takeout.google.com/settings/takeout). There is data available for several Google services, only the Gmail data is used here. Depending on the ammount of data the request can take several hours. Once notified that the file is ready to be downloaded, the data will come in a `mailbox` format. After importing some useful modules we can explore and clean the data.

In [1]:
import mailbox
import pandas as pd
import csv
import unicodedata
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(rc={'figure.facecolor':'white'})
import dateparser

from time import time

## 1. Data preprocessing
We begin the analysis by saving the data file `gmail_data.mbox` in a directory called `data` (for privacy reason, this file is unavailable in the Github repository). The file can be loaded using the `mailbox` module

In [6]:
dir_path = 'data/'
filename = 'gmail_data.mbox'
file_path = dir_path + filename
mbox = mailbox.mbox(path_data)
print('samples:', len(mbox))

The file contains samples. Even though these are mostly email messages, many other entry types are counted, such as drafts and chats. These can be removed by filtering by Gmail label. The file contains the following labels

In [7]:
# print labels
for i, key in enumerate(mbox[0].keys()):
    print(i+1, key)

We find that there are several section of little interest. In order to avoid loading unnecessary information, we can extract the fields of interest and put them into a `pandas` dataframe for further processing. We are interested in the following fields: `subject`, `from`, `to`, `date`, and `Gmail label`.

In [8]:
t0 = time()
subject = []
from_ = []
to = []
date = []
label = []
for i, message in enumerate(mbox):
    try:
        if i%2000 == 0:
            print(i, end=' ')
        subject.append(message['subject'])
        from_.append(message['from'])
        to.append(message['to'])
        date.append(message['date'])
        label.append(message['X-Gmail-Labels'])
    except:
        print(i, end=' ')
        print('subject', subject[i])
        print('from', from_[i])
        print('to', to[i])
        print('date', date[i])
        print('label', label[i])
print('\ntime: {:.1f} min'.format((time()-t0)/60))
print(len(subject))

In [11]:
df = pd.DataFrame()
df['subject'] = subject
df['from'] = from_
df['to'] = to
df['date'] = date
df['label'] = label

In [19]:
df[['subject', 'date', 'label']].head()

,subject,date,label
0,consulta DS,"Tue, 8 Aug 2017 11:20:05 +0200","Important,Sent"
1,Re: consulta DS,"Tue, 08 Aug 2017 10:35:39 +0100","Important,Inbox"
2,Re: consulta DS,"Tue, 08 Aug 2017 10:43:30 +0100","Important,Inbox"
3,Re: consulta DS,"Tue, 8 Aug 2017 13:16:37 +0200",Sent
4,NaN,"Sun, 19 Nov 2017 14:28:18 +0100","Important,Inbox"


My Gmail data contains mostly messages in English; however, there is plenty of Spanish and German, which introduce special characters that can lead to bugs. For this reason, special characters such as `ñ` and letters with accents and umlauts are converted or removed.

In [ ]:
def remove_accents(text):
    text = str(text)
    nfkd_norm = unicodedata.normalize('NFKD', text)
    text = nfkd_norm.encode('ASCII', 'ignore').decode('utf-8')
    return text

In [ ]:
df['subject'] = df['subject'].map(remove_accents)

After cleaning the `subject` field, we can get a general overview of the integrity of different fields using `df.info()`

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69988 entries, 0 to 69987
Data columns (total 5 columns):
subject    66828 non-null object
from       69988 non-null object
to         55762 non-null object
date       56735 non-null object
label      69604 non-null object
dtypes: object(5)
memory usage: 2.7+ MB


We find tha `date`, one of the most relevant fields, contains many null entries (mostly from chat entries)

In [29]:
df[df['date'].isnull()][['subject', 'to', 'date']].head()

,subject,to,date,label
16,None,NaN,NaN,Chat
26,None,NaN,NaN,Chat
90,None,NaN,NaN,Chat
91,None,NaN,NaN,Chat
92,None,NaN,NaN,Chat


The total number of null entries is

In [31]:
len(df[df['date'].isnull()])

56735

All these entries can be removed

In [32]:
# delete null rows
df = df[df['date'].notnull()]
df = df.reset_index(drop=True)

In [33]:
len(df)

56735

We can now focus on the `date` field, which is given as a `str`

In [34]:
df[['date']].head()

,date
0,"Tue, 8 Aug 2017 11:20:05 +0200"
1,"Tue, 08 Aug 2017 10:35:39 +0100"
2,"Tue, 08 Aug 2017 10:43:30 +0100"
3,"Tue, 8 Aug 2017 13:16:37 +0200"
4,"Sun, 19 Nov 2017 14:28:18 +0100"


Using the `dataparser` module we can conver these string dates into timestamps

In [40]:
dateparser.parse(df['date'][0])

datetime.datetime(2017, 8, 8, 11, 20, 5, tzinfo=<StaticTzInfo 'UTC\+02:00'>)

In [ ]:
t0 = time()
df['date'] = df['date'].map(dateparser.parse)
print('{:.1f}'.format((time() - t0)/60))

In [55]:
t0 = time()
date_lst = []
for i in range(len(df)):
    date_lst.append(dateparser.parse(df['date'][i]))
print('{:.1f} min'.format((time() - t0)/60))

28.6 min


In [61]:
df['date'] = date_lst

In [69]:
df[['subject', 'date', 'label']].head()

,subject,date,label
0,consulta DS,2017-08-08 11:20:05+02:00,"Important,Sent"
1,Re: consulta DS,2017-08-08 10:35:39+01:00,"Important,Inbox"
2,Re: consulta DS,2017-08-08 10:43:30+01:00,"Important,Inbox"
3,Re: consulta DS,2017-08-08 13:16:37+02:00,Sent
4,NaN,2017-11-19 14:28:18+01:00,"Important,Inbox"


At this point, and given the time used for cleaning the data file it is a good idea to export it as a `csv` file for future use without the need of redoing the preprocessing above.

In [70]:
df.to_csv('data/gmail_data2.csv', encoding='utf-8', index=False)

In [18]:
df = pd.read_csv('data/gmail_data.csv')

In [27]:
def dow(date):
    days=["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
    dayNumber=date.weekday()
    return days[dayNumber]

In [60]:
df['dow'] = df['date'].map(str2ts)

ValueError: time data ' 8 Aug 2017 13:16:37 ' does not match format ' %d %b %Y %I:%M:%S '

In [31]:
dow_tmp = []
for i in range(len(df)):
    try:
        dow_tmp.append(dow(df['date'][i]))
    except:
        print(df['date'][i], end=' ')

NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT NaT 

In [33]:
len(dow_tmp), len(df)

(56185, 56556)

In [71]:
# df['dow'].value_counts().plot(kind='bar'); #.index.tolist()

In [18]:
plt.hist(x);

NameError: name 'x' is not defined